## 参考：[My best single model - simple CNN - LB 0.1541](https://www.kaggle.com/jirivrany/my-best-single-model-simple-cnn-lb-0-1541)

In [1]:
import pandas as pd 
import numpy as np 
import cv2 # Used to manipulated the images 
seed = 1234
np.random.seed(seed) # The seed I used - pick your own or comment out for a random seed. A constant seed allows for better comparisons though

# Kfold
from sklearn.model_selection import StratifiedKFold

# Import Keras 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.optimizers import SGD

Using TensorFlow backend.


In [6]:
def get_scaled_imgs(df):
    """
    basic function for reshaping and rescaling data as images
    """
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
#         band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)

        band_3 = np.zeros((75,75))
        
        subt = abs(band_1-band_2)
        W1 = subt/subt.max()
        W2=1-W1
        band_3=W1 * band_1 + W2 * band_2
        
        '''
        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())
        '''

#         imgs.append(np.dstack((a, b, c)))
        imgs.append(np.dstack((band_1, band_2, band_3)))

    return np.array(imgs)

In [3]:
def get_more_images(imgs):
    """
    augmentation for more data
    """    


    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        a=imgs[i,:,:,0]
        b=imgs[i,:,:,1]
        c=imgs[i,:,:,2]
        
        av=cv2.flip(a,1)
        ah=cv2.flip(a,0)
        bv=cv2.flip(b,1)
        bh=cv2.flip(b,0)
        cv=cv2.flip(c,1)
        ch=cv2.flip(c,0)
        
        vert_flip_imgs.append(np.dstack((av, bv, cv)))
        hori_flip_imgs.append(np.dstack((ah, bh, ch)))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h))
    
    return more_images

In [8]:
def get_model():
    
    """
    Keras Sequential model

    """
    
    model=Sequential()
    
    # Conv block 1
    model.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu' ))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu' ))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
   
    # Conv block 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
   
    # Conv block 3
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
   
    #Conv block 4
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
   
    # Flatten before dense
    model.add(Flatten())

    #Dense 1
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.4))

    #Dense 2
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))

    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
#     optimizer = Adam(lr=0.0001, decay=0.0)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    return model

In [10]:
# Training Data
df_train = pd.read_json('./Data/train.json') # this is a dataframe


Xtrain = get_scaled_imgs(df_train)
Ytrain = np.array(df_train['is_iceberg'])
df_train.inc_angle = df_train.inc_angle.replace('na',0)
idx_tr = np.where(df_train.inc_angle>0)

Ytrain = Ytrain[idx_tr[0]]
Xtrain = Xtrain[idx_tr[0],...]

Xtr_more = get_more_images(Xtrain) 
Ytr_more = np.concatenate((Ytrain,Ytrain,Ytrain))

# K fold CV training
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
for fold_n, (train, test) in enumerate(kfold.split(Xtr_more, Ytr_more)):
    print("FOLD nr: ", fold_n)
    model = get_model()
    
    MODEL_FILE = './model/mdl_simple_k{}_wght.hdf5'.format(fold_n)
    batch_size = 16
    mcp_save = ModelCheckpoint(MODEL_FILE, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=15, verbose=1, epsilon=1e-4, mode='min')

    # set the epochs to 30 before training on your GPU
    model.fit(Xtr_more[train], Ytr_more[train],
        batch_size=batch_size,
        epochs=30,
        verbose=1,
        validation_data=(Xtr_more[test], Ytr_more[test]),
        callbacks=[mcp_save, reduce_lr_loss])
    
    model.load_weights(filepath = MODEL_FILE)

    score = model.evaluate(Xtr_more[test], Ytr_more[test], verbose=1)
    print('\n Val score:', score[0])
    print('\n Val accuracy:', score[1])

    SUBMISSION = './model/sub_simple_v1_{}.csv'.format(fold_n)

    df_test = pd.read_json('./Data/test.json')
    df_test.inc_angle = df_test.inc_angle.replace('na',0)
    Xtest = (get_scaled_imgs(df_test))
    pred_test = model.predict(Xtest)

    submission = pd.DataFrame({'id': df_test["id"], 'is_iceberg': pred_test.reshape((pred_test.shape[0]))})
    print(submission.head(10))

    submission.to_csv(SUBMISSION, index=False)
    print("submission saved")

FOLD nr:  0
Train on 3971 samples, validate on 442 samples
Epoch 1/30
3971/3971 [==============================] - 288s - loss: 0.6529 - acc: 0.5842 - val_loss: 0.5789 - val_acc: 0.6968
Epoch 2/30
3971/3971 [==============================] - 262s - loss: 0.4837 - acc: 0.7711 - val_loss: 0.4123 - val_acc: 0.8054
Epoch 3/30
3971/3971 [==============================] - 265s - loss: 0.3949 - acc: 0.8267 - val_loss: 0.5341 - val_acc: 0.7534
Epoch 4/30
3971/3971 [==============================] - 267s - loss: 0.3678 - acc: 0.8406 - val_loss: 0.3660 - val_acc: 0.8281
Epoch 5/30
3971/3971 [==============================] - 264s - loss: 0.3343 - acc: 0.8572 - val_loss: 0.2759 - val_acc: 0.8824
Epoch 6/30
3971/3971 [==============================] - 259s - loss: 0.2889 - acc: 0.8784 - val_loss: 0.2722 - val_acc: 0.8914
Epoch 7/30
3971/3971 [==============================] - 259s - loss: 0.2859 - acc: 0.8819 - val_loss: 0.2657 - val_acc: 0.8891
Epoch 8/30
3971/3971 [==============================

MemoryError: 

In [ ]:
wdir = './submission/'
stacked_1 = pd.read_csv(wdir + 'sub_simple_v1_0.csv')
stacked_2 = pd.read_csv(wdir + 'sub_simple_v1_1.csv')
stacked_3 = pd.read_csv(wdir + 'sub_simple_v1_2.csv')
stacked_4 = pd.read_csv(wdir + 'sub_simple_v1_3.csv')
stacked_5 = pd.read_csv(wdir + 'sub_simple_v1_4.csv')
stacked_6 = pd.read_csv(wdir + 'sub_simple_v1_5.csv')
stacked_7 = pd.read_csv(wdir + 'sub_simple_v1_6.csv')
stacked_8 = pd.read_csv(wdir + 'sub_simple_v1_7.csv')
stacked_9 = pd.read_csv(wdir + 'sub_simple_v1_8.csv')
stacked_10 = pd.read_csv(wdir + 'sub_simple_v1_9.csv')
sub = pd.DataFrame()
sub['id'] = stacked_1['id']
sub['is_iceberg'] = np.exp(np.mean(
    [
        stacked_1['is_iceberg'].apply(lambda x: np.log(x)),
        stacked_2['is_iceberg'].apply(lambda x: np.log(x)),
        stacked_3['is_iceberg'].apply(lambda x: np.log(x)),
        stacked_4['is_iceberg'].apply(lambda x: np.log(x)),
        stacked_5['is_iceberg'].apply(lambda x: np.log(x)),
        stacked_6['is_iceberg'].apply(lambda x: np.log(x)),
        stacked_7['is_iceberg'].apply(lambda x: np.log(x)),
        stacked_8['is_iceberg'].apply(lambda x: np.log(x)),
        stacked_9['is_iceberg'].apply(lambda x: np.log(x)),
        stacked_10['is_iceberg'].apply(lambda x: np.log(x)),
        ], axis=0))

sub.to_csv(wdir + 'final_ensemble13.1.csv', index=False, float_format='%.6f') 